# Model Testing

In [ ]:
import requests
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Replace with your actual IBM Cloud API key
API_KEY = "Your_Api_Key"

# Get access token
token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'}
)
mltoken = token_response.json()["access_token"]
header = {'Content-Type': 'application/json', 'Authorization': f'Bearer {mltoken}'}

# Payload: test data including known Target labels
payload_scoring = {
    "input_data": [
        {
            "fields": ["Product ID", "Type", "Air temperature [K]", "Process temperature [K]",
                       "Rotational speed [rpm]", "Torque [Nm]", "Tool wear [min]", "Target"],
            "values": [
                ["M18096","M",300.8,309.4,1342,62.4,113,1], # Heat Dissipation Failure
                ["L47184","L",298.2,308.7,1408,40,9,0], # No Failure
                ["L47428","L",298,308.3,1362,56.8,216,1], # Overstrain Failure
                ["L47622","L",297.4,308.5,1399,61.5,61,1], # Power Failure
                ["H32866","H",301.6,310.5,1602,32.3,2,0], # Random Failures
                ["L50791","L",301.7,310.9,1405,46.4,207,1] # Tool Wear Failure
                
            ]
        }
    ]
}

# Send request to AutoAI model
response_scoring = requests.post(
    'deployment_url,
    json=payload_scoring,
    headers=header
)


# Handle response
try:
    result = response_scoring.json()
    predictions = result['predictions'][0]['values']
    
    pred_labels = []
    print("\nPredictions:")
    for i, (label, probs) in enumerate(predictions):
        print(f"{i+1}. Predicted: {label} | Probabilities: {probs} ")
        
except ValueError:
    print(response_scoring.text)
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Predictions:
1. Predicted: Heat Dissipation Failure | Probabilities: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
2. Predicted: No Failure | Probabilities: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0] 
3. Predicted: Overstrain Failure | Probabilities: [0.0030303031206130983, 0.0, 0.9969696998596191, 0.0, 0.0, -2.9802322831784522e-09] 
4. Predicted: Power Failure | Probabilities: [0.0, 0.0, 0.1, 0.9, 0.0, 0.0] 
5. Predicted: Random Failures | Probabilities: [0.0, 0.4, 0.0, 0.0, 0.6000000000000001, 0.0] 
6. Predicted: Tool Wear Failure | Probabilities: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0] 
